<a href="https://colab.research.google.com/github/kaarthikegt/BackTestSim/blob/master/Data_Migration_Resume_Parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install spacy_transformers
!pip install -U spacy

In [28]:
import json
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
spacy_version = spacy.__version__
gpu_type = !nvidia-smi

print(spacy_version)
print(gpu_type)

3.7.2
['Tue Jan 16 07:52:53 2024       ', '+---------------------------------------------------------------------------------------+', '| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |', '|-----------------------------------------+----------------------+----------------------+', '| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |', '| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |', '|                                         |                      |               MIG M. |', '|=========================================+======================+======================|', '|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |', '| N/A   41C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |', '|                                         |                      |                  N/A |', '+--------------------------

In [29]:
!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

fatal: destination path 'CV-Parsing-using-Spacy-3' already exists and is not an empty directory.


In [30]:
cv_data = json.load(open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json'))
len(cv_data)

200

In [31]:
cv_data[3]

["Ijas Nizamuddin Associate Consultant - State Street  Irinchayam B.O, Kerala - Email me on Indeed: indeed.com/r/Ijas- Nizamuddin/6748d77f76f94eed  With close to 3 years of experience in IT industry, I have had excellent exposure to design, development and implementation of Client Server Applications in various domains such as Banking and Finance concepts. I have been involved in various software Development projects in Open System environment.  WORK EXPERIENCE  Associate Consultant  Oracle Corporation -  June 2011 to Present  State Street Global Advisors (SSgA) is the asset management business of State Street Corporation, one of the world's leading providers of financial services to institutional investors1, with a heritage dating back over two centuries. Backed by the strength and stability of the State Street organization, SSgA makes continual investments in asset management and client service platform, resulting in a client-focused, solutions-driven orientation .BrokerViews is the 

In [32]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config.cfg


2024-01-16 07:52:57.136375: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-16 07:52:57.136422: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-16 07:52:57.137665: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-16 07:52:58.176083: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/CV-Parsing-using-Spacy-3/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.

In [52]:
def get_spacy_doc(file, data):
    nlp = spacy.blank('en')
    db = DocBin()

    for text, annot in tqdm(data):
      doc = nlp.make_doc(text)
      annot = annot['entities']

      ents = []
      entity_indices = []

      for start, end, label in annot:
        skip_entity = False
        for idx in range(start, end):
          if idx in entity_indices:
            skip_entity = True
            break
        if skip_entity == True:
          continue

        entity_indices = entity_indices + list(range(start, end))

        try:
          span = doc.char_span(start, end, label=label, alignment_mode='strict')
        except:
          continue


        if span is None:
          err_data = str([start, end]) + "    " + str(text) + "\n"
          file.write(err_data)

        else:
          ents.append(span)

      try:
        doc.ents = ents
        db.add(doc)
      except:
        pass
    return db

In [46]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size = 0.3)

In [47]:
len(train), len(test)

(140, 60)

In [61]:
filep=open('error.txt', 'w')
db_for_train = get_spacy_doc(filep, train)
db_for_train.to_disk('train_data.spacy')

db_for_test = get_spacy_doc(filep, test)
db_for_test.to_disk('test_data.spacy')

filep.close()


100%|██████████| 60/60 [00:00<00:00, 90.78it/s]


In [62]:
print(len(db_for_train.tokens))
print(len(db_for_test.tokens))

140
60


In [66]:
# train the model using training data
!python -m spacy train /content/CV-Parsing-using-Spacy-3/data/training/config.cfg --output ./output --paths.train /content/train_data.spacy --paths.dev /content/test_data.spacy --gpu-id 0

2024-01-16 09:10:28.263408: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-16 09:10:28.263465: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-16 09:10:28.265104: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-16 09:10:29.819426: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly init

In [72]:
## Model Test
nlp = spacy.load('/content/output/model-best')
doc = nlp('my name is Manjoj Vel. I worked at CompuVision. I have 10 years of experience')
for ent in doc.ents:
  print(ent.text , "   >>>>", ent.label_)